In [47]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gensim
from gensim import models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [48]:
with open('friends_train.json',encoding='cp1252') as f:
  emotion_train = json.load(f)

with open('friends_dev.json',encoding='cp1252') as f:
  emotion_dev = json.load(f)

with open('friends_test.json',encoding='cp1252') as f:
  emotion_test = json.load(f)



In [49]:
train_data = pd.DataFrame(columns=['i_dialog','i_utterance','speaker','utterance','emotion'])
test_data = pd.DataFrame(columns=['i_dialog','i_utterance','speaker','utterance','emotion'])
kaggle_data = pd.read_csv('en_data.csv',encoding='cp1252')

for i_dialog,scen in enumerate(emotion_train):    
    for i_utterance,dialog in enumerate(scen):        
        train_data = train_data.append({'i_dialog':i_dialog,'i_utterance':i_utterance,'speaker':dialog['speaker'],'utterance':dialog['utterance'],'emotion':dialog['emotion']}, ignore_index=True)

for i_dialog,scen in enumerate(emotion_dev):    
    for i_utterance,dialog in enumerate(scen):        
        train_data = train_data.append({'i_dialog':i_dialog,'i_utterance':i_utterance,'speaker':dialog['speaker'],'utterance':dialog['utterance'],'emotion':dialog['emotion']}, ignore_index=True)


for i_dialog,scen in enumerate(emotion_test):
    # before_utterance = ''
    for i_utterance,dialog in enumerate(scen):        
        test_data = test_data.append({'i_dialog':i_dialog,'i_utterance':i_utterance,'speaker':dialog['speaker'],'utterance':dialog['utterance'],'emotion':dialog['emotion']}, ignore_index=True)
        # test_data = test_data.append({'i_dialog':i_dialog,'i_utterance':i_utterance,'speaker':dialog['speaker'],'utterance':before_utterance+' '+dialog['utterance'],'emotion':dialog['emotion']}, ignore_index=True)
        # before_utterance = dialog['utterance']

    

In [50]:
X_train = []
for row in train_data['utterance']:   
    X_train.append(gensim.utils.simple_preprocess(row))


X_test = []
for row in test_data['utterance']:   
    X_test.append(gensim.utils.simple_preprocess(row))


X_kaggle = []
for row in kaggle_data['utterance']:   
    X_kaggle.append(gensim.utils.simple_preprocess(row))    

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


In [51]:
vocab_size = len(tokenizer.word_index)
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [52]:
y_train = []
for a in train_data['emotion']:
    if a=='neutral':
        y_train = np.append(y_train, 1 )        
    elif a=='joy':
        y_train = np.append(y_train, 2)
    elif a=='sadness':
        y_train = np.append(y_train, 3)
    elif a=='fear':
        y_train = np.append(y_train, 4)
    elif a=='anger':
        y_train = np.append(y_train, 5)
    elif a=='suprise':
        y_train = np.append(y_train, 6)
    elif a=='disgust':
        y_train = np.append(y_train, 7)
    elif a=='non-neutral':
        y_train = np.append(y_train, 8)
    elif a=='surprise':
        y_train = np.append(y_train, 0)
y_train = to_categorical(y_train)



In [53]:
y_test = []
for a in test_data['emotion']:
    if a=='neutral':
        y_test = np.append(y_test, 1 )        
    elif a=='joy':
        y_test = np.append(y_test, 2)
    elif a=='sadness':
        y_test = np.append(y_test, 3)
    elif a=='fear':
        y_test = np.append(y_test, 4)
    elif a=='anger':
        y_test = np.append(y_test, 5)
    elif a=='suprise':
        y_test = np.append(y_test, 6)
    elif a=='disgust':
        y_test = np.append(y_test, 7)
    elif a=='non-neutral':
        y_test = np.append(y_test, 8)
    elif a=='surprise':
        y_test = np.append(y_test, 0)
y_test = to_categorical(y_test)

In [54]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이 : 65
리뷰의 평균 길이 : 7.389896924780646


In [55]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [56]:
max_len = 22
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 22 이하인 샘플의 비율: 98.06627481046085


In [57]:

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [68]:
model = Sequential()
model.add(Embedding(vocab_size, 15))
model.add(LSTM(128))
#model.add(Dense(9, activation='relu'))
model.add(Dense(9, activation='softmax'))
#model.add(Dense(9, activation='sigmoid'))



In [69]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(X_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(X_test, y_test))


Epoch 1/30
91/92 [============================>.] - ETA: 0s - loss: 1.6770 - acc: 0.4451
Epoch 00001: val_acc improved from -inf to 0.46563, saving model to best_model.h5
92/92 [==============================] - 4s 45ms/step - loss: 1.6758 - acc: 0.4450 - val_loss: 1.5702 - val_acc: 0.4656
Epoch 2/30
92/92 [==============================] - ETA: 0s - loss: 1.5900 - acc: 0.4467
Epoch 00002: val_acc improved from 0.46563 to 0.46599, saving model to best_model.h5
92/92 [==============================] - 3s 38ms/step - loss: 1.5900 - acc: 0.4467 - val_loss: 1.5497 - val_acc: 0.4660
Epoch 3/30
91/92 [============================>.] - ETA: 0s - loss: 1.5135 - acc: 0.4609
Epoch 00003: val_acc improved from 0.46599 to 0.48770, saving model to best_model.h5
92/92 [==============================] - 3s 38ms/step - loss: 1.5138 - acc: 0.4612 - val_loss: 1.4956 - val_acc: 0.4877
Epoch 4/30
91/92 [============================>.] - ETA: 0s - loss: 1.3992 - acc: 0.5058
Epoch 00004: val_acc did not imp

In [70]:

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

87/87 [==============================] - 1s 6ms/step - loss: 1.4956 - acc: 0.4877

 테스트 정확도: 0.4877


In [ ]:

X_kaggle = []
for row in kaggle_data['utterance']:   
    X_kaggle.append(gensim.utils.simple_preprocess(row))    
X_kaggle = tokenizer.texts_to_sequences(X_kaggle)
X_kaggle = pad_sequences(X_kaggle, maxlen = max_len)


In [ ]:
kaggle_result = pd.DataFrame(columns=['Id','Predicted'])
i=0
for p in loaded_model.predict(X_kaggle):
    emotion = np.argmax(p)
    if emotion == 1:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'neutral'}, ignore_index=True)
    elif emotion == 2: 
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'joy'}, ignore_index=True)
    elif emotion == 3:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'sadness'}, ignore_index=True)
    elif emotion == 4:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'fear'}, ignore_index=True)
    elif emotion == 5:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'anger'}, ignore_index=True)
    elif emotion == 6:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'suprise'}, ignore_index=True)
    elif emotion == 7:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'disgust'}, ignore_index=True)
    elif emotion == 8:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'non-neutral'}, ignore_index=True)
    elif emotion == 0:
        kaggle_result = kaggle_result.append({'Id':i,'Predicted':'surprise'}, ignore_index=True)
    i=i+1

kaggle_result.to_csv('2조.csv', index=False)   
